# How to Compute the Conjunctive Normal Form

In [ ]:
import propLogParser as plp

In [ ]:
def eliminateBiconditional(f):
    "Eliminate the logical operator '↔' from the formula f."
    if isinstance(f, str): 
        return f
    if f[0] == '↔':
        g, h = f[1:]
        ge   = eliminateBiconditional(g)
        he   = eliminateBiconditional(h)
        return ('∧', ('→', ge, he), ('→', he, ge))
    if f[0] == '¬':
        g  = f[1]
        ge = eliminateBiconditional(g)
        return ('¬', g)
    else:
        op, g, h = f
        ge       = eliminateBiconditional(g)
        he       = eliminateBiconditional(h)
        return (op, ge, he)

In [ ]:
def eliminateConditional(f):
    "Eliminate the logical operator '→' from f."
    if isinstance(f, str): 
        return f
    if f[0] == '→':
        g, h = f[1:]
        ge   = eliminateConditional(g)
        he   = eliminateConditional(h)
        return ('∨', ('¬', ge), he)
    if f[0] == '¬':
        g  = f[1]
        ge = eliminateConditional(g)
        return ('¬', g)
    else:
        op, g, h = f
        ge       = eliminateConditional(g)
        he       = eliminateConditional(h)
        return (op, ge, he)

In [ ]:
def nnf(f):
    "Compute the negation normal form of f."
    if isinstance(f, str): 
        return f
    if f[0] == '¬':
        g = f[1]
        return neg(g)
    if f[0] == '∧':
        g, h = f[1:]
        return ('∧', nnf(g), nnf(h))
    if f[0] == '∨':
        g, h = f[1:]
        return ('∨', nnf(g), nnf(h))

def neg(f):
    "Compute the negation normal form of ¬f."
    if isinstance(f, str): 
        return ('¬', f)
    if f[0] == '¬':
        g = f[1]
        return nnf(g)
    if f[0] == '∧':
        g, h = f[1:]
        return ('∨', nnf(g), nnf(h))
    if f[0] == '∨':
        g, h = f[1:]
        return ('∧', nnf(g), nnf(h))

In [ ]:
def cnf(f):
    if isinstance(f, str): 
        return { frozenset({f}) }
    if f[0] == '¬':
        return { frozenset({f}) }
    if f[0] == '∧':
        g, h = f[1:]
        return cnf(g) | cnf(h)
    if f[0] == '∨':
        g, h = f[1:]
        return { k1 | k2 for k1 in cnf(g) for k2 in cnf(h) }

In [ ]:
def isTrivial(Clause):
    return any(('¬', p) in Clause for p in Clause)

def simplify(Clauses):
    return { C for C in Clauses if not isTrivial(C) }

In [ ]:
def normalize (f):
    n1 = eliminateBiconditional(f)
    n2 = eliminateConditional(n1)
    n3 = nnf(n2)
    n4 = cnf(n3)
    return simplify(n4)

In [ ]:
def test(s):
    f = plp.LogicParser(s).parse()
    print(f'The knf of {s} is:')
    print(prettify(normalize(f)))

In [ ]:
def prettify(M):
    """Turn the set of frozen sets M into a string that looks like a set of sets.
       M is assumed to be the power set of some set.
    """
    if M == set():
        return '{}'
    result = "{"
    for A in M:
        if A == set(): 
            result += "{}, "
        result += str(set(A)) + ", " # A is converted from a frozen set to a set
    result = result[:-2] # remove the trailing substring ", "
    result += "}"
    return result

In [ ]:
test('¬(a ∧ b) ↔ ¬a ∨ ¬b')

In [ ]:
test('(a → b) ↔ (¬a ∧ ¬b)')

In [ ]:
test('(p ∧ q → r) ∨ ¬r → ¬p')